In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from google.colab import files
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_excel(file_name)
data.head()

Saving flag dataset compressed.xlsx to flag dataset compressed.xlsx


,L,Lg1,Lg,Wg1,Wg2,Ls,Ws1,Wp,Frequency (GHz)
0,25.0,8.6,3.0,15.0,4.0,12.1,2.5,12.0,2.0263
1,25.0,8.6,3.1,15.0,4.0,12.1,2.5,12.0,2.0400
2,25.0,8.6,3.2,15.0,4.0,12.1,2.5,12.0,2.0263
3,25.0,8.6,3.3,15.0,4.0,12.1,2.5,12.0,2.0950
4,25.0,8.6,3.4,15.0,4.0,12.1,2.5,12.0,2.0950


In [ ]:
X = data.drop(columns=['Frequency (GHz)'])
y = data['Frequency (GHz)']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)
print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")

Training set size: 292
Test set size: 33


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# **TRAINING SVR WITHOUT GENETIC ALGORITHM**

In [ ]:
#training the model
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train_scaled, y_train)

SVR()

In [ ]:
# Predicting on the test set
y_pred = svr_model.predict(X_test_scaled)

# Calculating evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")


Mean Squared Error (MSE): 0.00444036647949601
R² Score: 0.8915382345318348


In [ ]:
import numpy as np
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = svr_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.1199731380426377


In [ ]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = svr_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.4008615092853716


## **IMPLEMENTING GENETIC ALGORITHM**

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.9 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import cross_val_score
from deap import base, creator, tools, algorithms
import numpy as np
# Defining the fitness function for GA
def fitness_function(params):
    C, epsilon, gamma = params
    if C <= 0 or epsilon <= 0 or gamma <= 0:  # Ensure positive values
        return float('inf'),  # High fitness value for invalid parameters
    svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
    scores = cross_val_score(svr, X_train_scaled, y_train, cv=3, scoring='neg_mean_squared_error')
    return -np.mean(scores),  # Minimize negative MSE


In [ ]:
# Create the individual and population
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Define attribute initialization ranges
toolbox.register("attr_C", np.random.uniform, 0.1, 100)  # C: [0.1, 100]
toolbox.register("attr_epsilon", np.random.uniform, 0.01, 1)  # epsilon: [0.01, 1]
toolbox.register("attr_gamma", np.random.uniform, 0.0001, 1)  # gamma: [0.0001, 1]

# Define individual and population
toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (toolbox.attr_C, toolbox.attr_epsilon, toolbox.attr_gamma),
    n=1,
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Define GA operators
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)

# Custom mutation to enforce valid ranges
def custom_mutation(individual, indpb):
    individual[0] = max(0.1, min(100, individual[0]))  # C: [0.1, 100]
    individual[1] = max(0.01, min(1, individual[1]))  # epsilon: [0.01, 1]
    individual[2] = max(0.0001, min(1, individual[2]))  # gamma: [0.0001, 1]
    return individual,

toolbox.register("mutate", custom_mutation, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", fitness_function)


In [ ]:
# Create an initial population
population = toolbox.population(n=30)  # Population size
ngen = 50                              # Number of generations
cxpb = 0.7                             # Crossover probability
mutpb = 0.2                            # Mutation probability

# Run the genetic algorithm
result, logbook = algorithms.eaSimple(population, toolbox, cxpb, mutpb, ngen, verbose=True)

# Extract the best individual
best_individual = tools.selBest(population, k=1)[0]
print("Best Parameters (C, epsilon, gamma):", best_individual)


gen	nevals
0  	30    
1  	23    
2  	22    
3  	16    
4  	20    
5  	25    
6  	22    
7  	25    
8  	19    
9  	27    
10 	26    
11 	15    
12 	28    
13 	24    
14 	27    
15 	27    
16 	27    
17 	18    
18 	23    
19 	21    
20 	23    
21 	21    
22 	22    
23 	20    
24 	23    
25 	24    
26 	17    
27 	22    
28 	24    
29 	22    
30 	27    
31 	26    
32 	20    
33 	23    
34 	26    
35 	17    
36 	26    
37 	27    
38 	25    
39 	23    
40 	21    
41 	28    
42 	23    
43 	25    
44 	23    
45 	24    
46 	28    
47 	25    
48 	20    
49 	23    
50 	22    
Best Parameters (C, epsilon, gamma): [2.710856267106335, 0.05502352328033054, 0.00545678185667421]


In [ ]:
# Train SVR with the best hyperparameters
optimized_svr = SVR(kernel='rbf', C=best_individual[0], epsilon=best_individual[1], gamma=best_individual[2])
optimized_svr.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_optimized = optimized_svr.predict(X_test_scaled)

# Evaluate
mse_optimized = mean_squared_error(y_test, y_pred_optimized)
r2_optimized = r2_score(y_test, y_pred_optimized)

print(f"Optimized Mean Squared Error (MSE): {mse_optimized}")
print(f"Optimized R² Score: {r2_optimized}")

Optimized Mean Squared Error (MSE): 0.0038562771318270575
Optimized R² Score: 0.9058053816540053


In [ ]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = optimized_svr.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.071253760590624


In [ ]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = optimized_svr.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.34436288905146


# **RANDOM FOREST WITHOUT GENETIC ALOGORITHM**


In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=5)
rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

Mean Squared Error (MSE): 0.0038633140444048063
R² Score: 0.9056334958501753


In [ ]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = rf_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.0330199999999987


In [ ]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = rf_model.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.3576580000000007


# **Random Forest with GA**

In [ ]:
def rf_fitness(params):
  n_estimators, max_depth, min_samples_split = map(int, params)
  if n_estimators < 50 or max_depth < 5 or min_samples_split < 2:
    return float('inf'),
  rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, random_state=7)
  return -cross_val_score(rf, X_train_scaled, y_train, cv=3, scoring='neg_mean_squared_error').mean(),

In [ ]:
from deap import creator

if "FitnessMin" not in creator.__dict__:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimize MSE

if "Individual" not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMin)


In [ ]:
toolbox = base.Toolbox()
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)
toolbox.register("attr_n_estimators", lambda: np.random.randint(50, 150))
toolbox.register("attr_max_depth", lambda: np.random.randint(5, 30))
toolbox.register("attr_min_samples_split", lambda: np.random.randint(2, 8))
toolbox.register("individual", tools.initIterate, creator.Individual, lambda: [toolbox.attr_n_estimators(), toolbox.attr_max_depth(), toolbox.attr_min_samples_split()])
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", rf_fitness)

In [ ]:
# Run GA
population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=60, verbose=True)
best = tools.selBest(population, k=1)[0]
print("Best Parameters:", best)

gen	nevals
0  	50    
1  	37    
2  	41    
3  	39    
4  	44    
5  	44    
6  	35    
7  	42    
8  	45    
9  	39    
10 	35    
11 	41    
12 	38    
13 	39    
14 	39    
15 	39    
16 	44    
17 	35    
18 	36    
19 	39    
20 	40    
21 	39    
22 	36    
23 	32    
24 	35    
25 	36    
26 	39    
27 	38    
28 	38    
29 	42    
30 	36    
31 	42    
32 	39    
33 	46    
34 	32    
35 	36    
36 	34    
37 	38    
38 	36    
39 	40    
40 	38    
41 	39    
42 	40    
43 	37    
44 	34    
45 	41    
46 	35    
47 	35    
48 	38    
49 	40    
50 	36    
51 	40    
52 	35    
53 	40    
54 	40    
55 	39    
56 	39    
57 	36    
58 	42    
59 	28    
60 	37    
Best Parameters: [75.63576220718254, 24.81293169930943, 3.5658232313876423]


In [ ]:
# Train optimized model
optimized_rf = RandomForestRegressor(n_estimators=int(best[0]), max_depth=int(best[1]), min_samples_split=int(best[2]), random_state=19)
optimized_rf.fit(X_train_scaled, y_train)

RandomForestRegressor(max_depth=24, min_samples_split=3, n_estimators=75,
                      random_state=19)

In [ ]:
# Evaluate
y_pred = optimized_rf.predict(X_test_scaled)
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
print(f"R²: {r2_score(y_test, y_pred)}")

MSE: 0.0038208716954943366
R²: 0.9066702057962354


In [ ]:
# Define the new input data
new_data = np.array([[25, 8.6, 3, 15, 4, 12.1, 2.5, 12]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = optimized_rf.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.0342651904761904


In [ ]:
# Define the new input data
new_data = np.array([[26, 8.8, 5.2, 14.8, 4, 12.2, 3, 10]])
new_data_df = pd.DataFrame(new_data, columns=X_train.columns)
# Scale the input data using the same scaler used for training
new_data_scaled = scaler.transform(new_data_df)
# Predict the frequency
predicted_frequency = optimized_rf.predict(new_data_scaled)
print(f"Predicted Frequency (GHz): {predicted_frequency[0]}")

Predicted Frequency (GHz): 2.348890355555555


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
# Actual frequency value
actual_frequency = np.array([2.0263])
# Predicted values from different models
predictions = {
    "SVR without GA": np.array([2.1199]),
    "SVR with GA": np.array([2.0547]),
    "RF without GA": np.array([2.0330]),
    "RF with GA": np.array([2.0342])
}
# Compute and display MSE for each model
for model, pred in predictions.items():
    mse = mean_squared_error(actual_frequency, pred)
    print(f"{model} - MSE: {mse:.6f}")

SVR without GA - MSE: 0.008761
SVR with GA - MSE: 0.000807
RF without GA - MSE: 0.000045
RF with GA - MSE: 0.000062
